# Getting the dataset
A script has been provided to download all the datasets required for running the below examples.
It will dowload and unzip the WikiQA Corpus and the Quora Duplicate Questions dataset.

In [ ]:
!python experimental_data/get_data.py

## Installing dependencies for running the Similarity Learning task

In [1]:
import os
import csv
import re
from gensim.models.experimental.drmm_tks import DRMM_TKS
from gensim.utils import simple_preprocess

Using TensorFlow backend.
2018-07-07 20:45:03,269 : INFO : 'pattern' package not found; tag filters are not available for English


## Data Format

We have to provide data in a format which is understood by the model.
The model understands sentences as a list of words. 
Further, we need to give a :
 1. Queries List
 2. Candidate Document List
 3. Correct Label List

1 is a list of list of words
2 and 3 is actually a list of list of list of words/ints

Example:
```
queries = ["When was Abraham Lincoln born ?".split(), 
            "When was the first World War ?".split()]
docs = [
		 ["Abraham Lincoln was the president of the United States of America".split(),
		 "He was born in 1809".split()],
		 ["The first world war was bad".split(),
		 "It was fought in 1914".split(),
		 "There were over a million deaths".split()]
       ]
labels = [[0,
           1],
		  [0,
           1,
           0]
          ]
```

## About the dataset : WikiQA

The WikiQA corpus is a set of question-answer pairs in which for every query there are several candidate documents of which none, one or more documents might be relevant.
Relevance is purely binary, i.e., 1: relavant, 0: not relevant

Sample data:

QuestionID | Question | DocumentID | DocumentTitle | SentenceID | Sentence | Label
-- | -- | -- | -- | -- | -- | --
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-0 | A partly submerged glacier cave on Perito Moreno Glacier . | 0
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-1 | The ice facade is approximately 60 m high | 0
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-2 | Ice formations in the Titlis glacier cave | 0
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-3 | A glacier cave is a cave formed within the ice of a glacier . | 1
Q1 | how are glacier caves formed? | D1 | Glacier cave | D1-4 | Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-0 | In physics , circular motion is a movement of an object along the circumference of a circle or rotation along a circular path. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-1 | It can be uniform, with constant angular rate of rotation (and constant speed), or non-uniform with a changing rate of rotation. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-2 | The rotation around a fixed axis of a three-dimensional body involves circular motion of its parts. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-3 | The equations of motion describe the movement of the center of mass of a body. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-4 | Examples of circular motion include: an artificial satellite orbiting the Earth at constant height, a stone which is tied to a rope and is being swung in circles, a car turning through a curve in a race track , an electron moving perpendicular to a uniform magnetic field , and a gear turning inside a mechanism. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-5 | Since the object's velocity vector is constantly changing direction, the moving object is undergoing acceleration by a centripetal force in the direction of the center of rotation. | 0
Q2 | How are the directions of the velocity and force vectors related in a circular motion | D2 | Circular motion | D2-6 | Without this acceleration, the object would move in a straight line, according to Newton's laws of motion . | 0

## Data Preprocessing
We need to take the above text and make it into `queries, docs, labels` form. For this, we will create an iterable object with the below class which will allow the data to be streamed into the model as the need arises.

In [2]:
class MyWikiIterable:
    """"Yields the next data point in the data set based on the `iter_type`
    
    Based on `iter_type` the object can yield the following:
        'query' : list of str words
        'doc' : list of docs
                    where a doc is a list of str words
        'label' : list of int
                  The relevance between adjacent queries and docs
    """

    def __init__(self, iter_type, fpath):
        """
        Parameters
        ----------
        iter_type : {'query', 'doc', 'label'}
            the type of iterable to be yielded
        fpath : str
            path to the dataset
        """

        # To map the `iter_type` to an index
        self.type_translator = {'query': 0, 'doc': 1, 'label': 2}
        self.iter_type = iter_type

        with open(fpath, encoding='utf8') as tsv_file:
            tsv_reader = csv.reader(tsv_file, delimiter='\t', quoting=csv.QUOTE_NONE)
            self.data_rows = []
            self.data_rows = [row for row in tsv_reader]

    def preprocess_sent(self, sent):
        """Utility function to lower, strip and tokenize each sentence
        Replace this function if you want to handle preprocessing differently"""

        return simple_preprocess(sent)

    def __iter__(self):
        # Defining some consants for .tsv reading
        # They represent the columns of the respective values
        QUESTION_ID_INDEX = 0
        QUESTION_INDEX = 1
        ANSWER_INDEX = 5
        LABEL_INDEX = 6


        # The group of documents and labels that belong to one question
        document_group = []
        label_group = []

        # Number of relevant documents per query
        n_relevant_docs = 0
        # Number of filtered docs (query-doc pairs which have zero relevant docs)
        n_filtered_docs = 0

        # The data
        queries = []
        docs = []
        labels = []

        # The code below goes through the data line by line
        # It checks the current document id with the next document id
        for i, line in enumerate(self.data_rows[1:], start=1):

            if i < len(self.data_rows) - 1:  # check if out of bounds might occur

                if self.data_rows[i][QUESTION_ID_INDEX] == self.data_rows[i + 1][QUESTION_ID_INDEX]:
                    document_group.append(self.preprocess_sent(self.data_rows[i][ANSWER_INDEX]))
                    label_group.append(int(self.data_rows[i][LABEL_INDEX]))
                    n_relevant_docs += int(self.data_rows[i][LABEL_INDEX])
                else:
                    document_group.append(self.preprocess_sent(self.data_rows[i][ANSWER_INDEX]))
                    label_group.append(int(self.data_rows[i][LABEL_INDEX]))
                    n_relevant_docs += int(self.data_rows[i][LABEL_INDEX])

                    if n_relevant_docs > 0:
                        docs.append(document_group)
                        labels.append(label_group)
                        queries.append(self.preprocess_sent(self.data_rows[i][QUESTION_INDEX]))

                        yield [queries[-1], document_group, label_group][self.type_translator[self.iter_type]]
                    else:
                        n_filtered_docs += 1

                    n_relevant_docs = 0
                    document_group = []
                    label_group = []

            else:
                # If we are on the last line
                document_group.append(self.preprocess_sent(self.data_rows[i][ANSWER_INDEX]))
                label_group.append(int(self.data_rows[i][LABEL_INDEX]))
                n_relevant_docs += int(self.data_rows[i][LABEL_INDEX])

                if n_relevant_docs > 0:
                    docs.append(document_group)
                    labels.append(label_group)
                    queries.append(self.preprocess_sent(self.data_rows[i][QUESTION_INDEX]))
                    yield [queries[-1], document_group, label_group][self.type_translator[self.iter_type]]
                else:
                    n_filtered_docs += 1
                    n_relevant_docs = 0
        print("Skipped ", n_filtered_docs)

In [3]:
l_iterable = MyWikiIterable('label', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-train.tsv'))
for l in l_iterable:
    print(l)
    if sum(l) == 0:
        bdfbv

[0, 0, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0]
[1, 1]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 0]
[0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[1, 0]
[

[0, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1]
[1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1]
[0, 0, 0, 1, 1, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0]
[0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0

Now, will use the class to create objects of the training iterable

In [4]:
q_iterable = MyWikiIterable('query', os.path.join( 'experimental_data', 'WikiQACorpus', 'WikiQA-train.tsv'))
d_iterable = MyWikiIterable('doc', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-train.tsv'))
l_iterable = MyWikiIterable('label', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-train.tsv'))

We will also initialize some validation iterables
Note: the path has `dev` in it

In [5]:
q_val_iterable = MyWikiIterable('query', os.path.join( 'experimental_data', 'WikiQACorpus', 'WikiQA-dev.tsv'))
d_val_iterable = MyWikiIterable('doc', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-dev.tsv'))
l_val_iterable = MyWikiIterable('label', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-dev.tsv'))

# Using word embeddings
We also need to get the word embeddings for the training. For this, we will use the Glove Embeddings.
Luckily, [gensim-data](https://github.com/RaRe-Technologies/gensim-data) provides an easy interface for it.

We will use the [KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) object that we for from gensim-data api and pass it as the `word_embedding` parameter in the model.

In [6]:
import gensim.downloader as api
kv_model = api.load("glove-wiki-gigaword-300")

2018-07-07 20:45:20,661 : INFO : loading projection weights from /home/aneeshj/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
2018-07-07 20:47:10,241 : INFO : loaded (400000, 300) matrix from /home/aneeshj/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz


# Training the Model
Now that we have the preprocessed extracted data and word embeddings, training the model just takes one line:

In [30]:
# Train the model
drmm_tks_model = DRMM_TKS(q_iterable, d_iterable, l_iterable, word_embedding=kv_model,
                                epochs=3, validation_data=[q_val_iterable, d_val_iterable, l_val_iterable])

2018-07-06 19:33:14,396 : INFO : Starting Vocab Build


Skipped  1245


2018-07-06 19:33:21,367 : INFO : Vocab Build Complete
2018-07-06 19:33:21,368 : INFO : Vocab Size is 18814
2018-07-06 19:33:21,369 : INFO : Building embedding index using KeyedVector pretrained word embeddings
2018-07-06 19:33:21,369 : INFO : The embeddings_index built from the given file has 400000 words of 300 dimensions
2018-07-06 19:33:21,370 : INFO : Building the Embedding Matrix for the model's Embedding Layer


Skipped  1245


2018-07-06 19:34:26,102 : INFO : There are 642 words out of 18814 (3.41%) not in the embeddings. Setting them to random
2018-07-06 19:34:26,104 : INFO : Adding additional words from the embedding file to embedding matrix
2018-07-06 19:35:07,562 : INFO : Normalizing the word embeddings
2018-07-06 19:36:17,499 : INFO : Embedding Matrix build complete. It now has shape (400644, 300)
2018-07-06 19:36:23,607 : INFO : Pad word has been set to index 400642
2018-07-06 19:36:25,849 : INFO : Unknown word has been set to index 400643
2018-07-06 19:36:26,071 : INFO : Embedding index build complete
2018-07-06 19:36:55,938 : INFO : Input is an iterable amd will be streamed
2018-07-06 19:38:26,868 : INFO : __________________________________________________________________________________________________
2018-07-06 19:38:28,230 : INFO : Layer (type)                    Output Shape         Param #     Connected to                     
2018-07-06 19:38:28,254 : INFO : ===================================

...

2018-07-06 19:41:24,602 : INFO : MAP: 0.60
2018-07-06 19:41:24,639 : INFO : nDCG@1 : 0.42
2018-07-06 19:41:24,644 : INFO : nDCG@3 : 0.60
2018-07-06 19:41:24,651 : INFO : nDCG@5 : 0.65
2018-07-06 19:41:24,656 : INFO : nDCG@10 : 0.69
2018-07-06 19:41:24,661 : INFO : nDCG@20 : 0.70


Epoch 2/3
888/900 [============================>.] - ETA: 1s - loss: 0.8346 - acc: 0.3258Skipped  1245
SAMPLA RE!!!!!!!!!!!!!!!!!! 8995
900/900 [==============================] - 84s 94ms/step - loss: 0.8354 - acc: 0.3273


2018-07-06 19:42:53,641 : INFO : MAP: 0.64
2018-07-06 19:42:53,646 : INFO : nDCG@1 : 0.48
2018-07-06 19:42:53,652 : INFO : nDCG@3 : 0.63
2018-07-06 19:42:53,660 : INFO : nDCG@5 : 0.68
2018-07-06 19:42:53,670 : INFO : nDCG@10 : 0.72
2018-07-06 19:42:53,677 : INFO : nDCG@20 : 0.73


Epoch 3/3
888/900 [============================>.] - ETA: 1s - loss: 0.8176 - acc: 0.3718Skipped  1245
SAMPLA RE!!!!!!!!!!!!!!!!!! 8995
900/900 [==============================] - 83s 92ms/step - loss: 0.8178 - acc: 0.3743


2018-07-06 19:44:21,704 : INFO : MAP: 0.64
2018-07-06 19:44:21,709 : INFO : nDCG@1 : 0.48
2018-07-06 19:44:21,719 : INFO : nDCG@3 : 0.63
2018-07-06 19:44:21,729 : INFO : nDCG@5 : 0.69
2018-07-06 19:44:21,736 : INFO : nDCG@10 : 0.72
2018-07-06 19:44:21,742 : INFO : nDCG@20 : 0.73


## Saving and loading the model
The trained model can be saved and loaded from memory for future use.

In [ ]:
drmm_tks_model.save('drmm_tks_model')

In [ ]:
del drmm_tks_model
drmm_tks_model = DRMM_TKS.load('drmm_tks_model')

## Testing the model on new data

The testing of the data can be done on completely unseen data using `model.predict(queries, docs)` where
queries: list of list of words
docs: list of list of list of words

In [19]:
queries = [simple_preprocess("how are glacier caves formed"),
           simple_preprocess("What is AWS")]

docs = [[simple_preprocess("A partly submerged glacier cave on Perito Moreno Glacier"),
        simple_preprocess("A glacier cave is a cave formed within the ice of a glacier")],
       [simple_preprocess("AWS stands for Amazon Web Services"),
        simple_preprocess("AWS was established in 2001"),
        simple_preprocess("It is a cloud service")]]

The predict function returns the similarity between a query-document pair in a list format

For example
```
queries = [q1, q2]
docs = [[d1_1, d1_2],
        [d2_1, d2_2, d2_3]]

model.predict(queries, docs)

Output
------
q1-d1_1 similarity
q1-d1_2 similarity
q2-d2_1 similarity
q2-d2_2 similarity
q2-d2_3 similarity
```

In [32]:
drmm_tks_model.predict(queries, docs)

2018-07-06 19:44:53,879 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-06 19:44:53,880 : INFO : Found 0 unknown words. Set them to unknown word index : 400643
2018-07-06 19:44:53,906 : INFO : Predictions in the format query, doc, similarity
2018-07-06 19:44:53,957 : INFO : ['how', 'are', 'glacier', 'caves', 'formed']	['partly', 'submerged', 'glacier', 'cave', 'on', 'perito', 'moreno', 'glacier']	0.60059035
2018-07-06 19:44:53,958 : INFO : ['how', 'are', 'glacier', 'caves', 'formed']	['glacier', 'cave', 'is', 'cave', 'formed', 'within', 'the', 'ice', 'of', 'glacier']	0.9258956
2018-07-06 19:44:53,959 : INFO : ['what', 'is', 'aws']	['aws', 'stands', 'for', 'amazon', 'web', 'services']	0.38946432
2018-07-06 19:44:53,959 : INFO : ['what', 'is', 'aws']	['aws', 'was', 'established', 'in']	0.37697053
2018-07-06 19:44:53,960 : INFO : ['what', 'is', 'aws']	['it', 'is', 'cloud', 'service']	0.5015697


array([[0.60059035],
       [0.9258956 ],
       [0.38946432],
       [0.37697053],
       [0.5015697 ]], dtype=float32)

In [21]:
for q, doc in zip(queries, docs):
    for d in doc:
        print(q, d, cosine_similarity(sent2vec(q, kv_model), sent2vec(d, kv_model)))

['how', 'are', 'glacier', 'caves', 'formed'] ['partly', 'submerged', 'glacier', 'cave', 'on', 'perito', 'moreno', 'glacier'] 0.65023756
['how', 'are', 'glacier', 'caves', 'formed'] ['glacier', 'cave', 'is', 'cave', 'formed', 'within', 'the', 'ice', 'of', 'glacier'] 0.8493792
['what', 'is', 'aws'] ['aws', 'stands', 'for', 'amazon', 'web', 'services'] 0.6288438
['what', 'is', 'aws'] ['aws', 'was', 'established', 'in'] 0.62425405
['what', 'is', 'aws'] ['it', 'is', 'cloud', 'service'] 0.69118476


As can be seen from the logs and results above, within each query-document group, the correct answer has the highest score

For example,
In the first group
```
['how', 'are', 'glacier', 'caves', 'formed'] ['partly', 'submerged', 'glacier', 'cave', 'on', 'perito', 'moreno', 'glacier']	0.53
['how', 'are', 'glacier', 'caves', 'formed'] ['glacier', 'cave', 'is', 'cave', 'formed', 'within', 'the', 'ice', 'of', 'glacier']	0.61
```

The correct answer, "glacier cave is cave ..." has the higher score as compared to the first answer
The second pair almost gets it right (WIP)

### Testing on a test set
We can pass a whole dataset and get evaluations based on that. Let's try with the test set of WikiQA Corpus

In [7]:
q_test_iterable = MyWikiIterable('query', os.path.join( 'experimental_data', 'WikiQACorpus', 'WikiQA-test.tsv'))
d_test_iterable = MyWikiIterable('doc', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-test.tsv'))
l_test_iterable = MyWikiIterable('label', os.path.join('experimental_data', 'WikiQACorpus', 'WikiQA-test.tsv'))

In [8]:
for q, doc, lab in zip(q_test_iterable, d_test_iterable, l_test_iterable):
    for d, l in zip(doc, lab):
        if len(q) ==0 or len(d)==0 :
            print(q,d,l)

Skipped  390


In [35]:
drmm_tks_model.evaluate(q_test_iterable, d_test_iterable, l_test_iterable)

Skipped  390


2018-07-06 19:45:14,550 : INFO : Found 21 unknown words. Set them to unknown word index : 400643
2018-07-06 19:45:24,857 : INFO : Found 249 unknown words. Set them to unknown word index : 400643
2018-07-06 19:45:34,533 : INFO : MAP: 0.62
2018-07-06 19:45:34,557 : INFO : nDCG@1 : 0.46
2018-07-06 19:45:34,571 : INFO : nDCG@3 : 0.60
2018-07-06 19:45:34,584 : INFO : nDCG@5 : 0.66
2018-07-06 19:45:34,602 : INFO : nDCG@10 : 0.70
2018-07-06 19:45:34,628 : INFO : nDCG@20 : 0.71


## Comparing DRMM TKS with other models

It would be good to get an idea of how our model works against some unsupervised models like word2vec and FastText.
For this, we will, given a query-document pair, we will get a vector for the query and document. We can get the similarity between them using the cosine similarity between their vectors.

### For word2vec


In [9]:
def cosine_similarity(vec1, vec2):
    if np.sum(vec1)==0 or np.sum(vec2)==0:
        halla
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

In [12]:
def sent2vec(sent, conv_dict):
    vec = []
    for word in sent:
        if word in conv_dict:
            vec.append(conv_dict[word])
    vec = np.array(vec)
    if np.sum(vec) == 0:
        print("Unk word.")
        return np.random.random((kv_model.vector_size,1))
    vec = np.mean(vec, axis=0)
    return vec

In [13]:
import numpy as np
sent2vec(simple_preprocess("aneesjh"), kv_model.wv)

Unk word.


/home/aneeshj/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


array([[0.06794987],
       [0.39836328],
       [0.78356935],
       [0.25651426],
       [0.79730968],
       [0.45313962],
       [0.61577883],
       [0.74594356],
       [0.03530246],
       [0.07807583],
       [0.38051976],
       [0.17457557],
       [0.02474486],
       [0.79364297],
       [0.10683484],
       [0.29434301],
       [0.5409396 ],
       [0.22821383],
       [0.56336627],
       [0.62904616],
       [0.29124177],
       [0.77077117],
       [0.30345319],
       [0.59490355],
       [0.68467406],
       [0.48006293],
       [0.49826349],
       [0.98917661],
       [0.41571828],
       [0.23730992],
       [0.01673158],
       [0.81632288],
       [0.99917037],
       [0.45786841],
       [0.35178298],
       [0.7749908 ],
       [0.97388189],
       [0.37443586],
       [0.2634219 ],
       [0.7453534 ],
       [0.84651779],
       [0.93776253],
       [0.56118499],
       [0.16238862],
       [0.27780575],
       [0.29107667],
       [0.03773087],
       [0.592

In [14]:
import numpy as np
from gensim.models.experimental import mapk, mean_ndcg

def eval_model(queries, docs, labels, model, _translate_user_data):

    long_label_list = []
    long_pred_list = []
    doc_lens = []
    
    for query, doc, label in zip(queries, docs, labels):
        for d, l in zip(doc, label):
            trans_q = _translate_user_data(query, model.wv)
            trans_d = _translate_user_data(d, model.wv)
            long_pred_list.append(cosine_similarity(trans_q, trans_d))
            long_label_list.append(l)
        doc_lens.append(len(doc))

#     check_doc_len = []
#     for d in docs:
#         check_doc_len.append(len(d))
        
    print(len(long_pred_list))
    print(len(long_label_list))
#     print(long_label_list)
    print(sum(doc_lens))
# #     print(doc_lens)
# #     print(check_doc_len)
    
# #     for i, q, d in zip(range(len(check_doc_len)), queries, docs):
# #         if doc_lens[i] != check_doc_len[i]:
# #             print(q, d)
# #             break

    predictions=long_pred_list
    Y_pred = []
    Y_true = []
    offset = 0
    for doc_size in doc_lens:
        Y_pred.append(predictions[offset: offset + doc_size])
        Y_true.append(long_label_list[offset: offset + doc_size])
        offset += doc_size

    print("MAP: %.2f" % mapk(Y_true, Y_pred))
    for k in [1, 3, 5, 10, 20]:
        print("nDCG@%d : %.2f"% (k, mean_ndcg(Y_true, Y_pred, k=k)))

In [15]:
eval_model(q_test_iterable, d_test_iterable, l_test_iterable, kv_model, sent2vec)

/home/aneeshj/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/home/aneeshj/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]


Skipped  390
2334
2334
2334
MAP: 0.63
nDCG@1 : 0.46
nDCG@3 : 0.62
nDCG@5 : 0.68
nDCG@10 : 0.71
nDCG@20 : 0.72


Let's compare that with our model

In [ ]:
drmm_tks_model.evaluate(q_test_iterable, d_test_iterable, l_test_iterable)

While the accuracy isn't any better, it is worse, this is still a Work In Progress and we hope to improve it further soon.